In [ ]:
import urllib.request
import json
import pandas as pd
import numpy as np
import sqlite3
import time
from loguru import logger

RESOURCE_ID = "f2e5503e-927d-4ad3-9500-4ab9e55deb59"
API_KEY = "1f958f4c-0cb6-44e3-a45a-d67fd237b3cb"

DB_NAME = "warsaw_ztm.db"
TABLE_NAME = "vehicles"

URL_BUS= f'https://api.um.warszawa.pl/api/action/busestrams_get/?resource_id=f2e5503e927d-4ad3-9500-4ab9e55deb59&apikey={API_KEY}&type=1'
URL_TRAM = f'https://api.um.warszawa.pl/api/action/busestrams_get/?resource_id=f2e5503e927d-4ad3-9500-4ab9e55deb59&apikey={API_KEY}&type=2'

In [ ]:
def create_db():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
            line TEXT,
            lon REAL,
            lat REAL,
            time TEXT,
            PRIMARY KEY (line, time)
        )
    ''')
    
    conn.commit()
    conn.close()

def fetch_data(url):
    with urllib.request.urlopen(url) as response:
        data = response.read().decode('utf-8')
        return json.loads(data)

def insert_data(data):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    for item in data.get("result", []):
        try:
            cursor.execute(f'''
                INSERT OR IGNORE INTO {TABLE_NAME} (line, lon, lat, time)
                VALUES (?, ?, ?, ?)
            ''', (
                item.get("Lines"),
                item.get("Lon"),
                item.get("Lat"),
                item.get("Time")
            ))
        except Exception as e:
            logger.error(f"Failed to insert data: {e}")

    conn.commit()
    conn.close()

def main():
    create_db()
    error_counter = 0
    
    while True:
        try:            
            json_data = fetch_data(URL_BUS)
            insert_data(json_data)
            
            json_data = fetch_data(URL_TRAM)
            insert_data(json_data)
            
            time.sleep(4)
        
        except Exception as e:
            error_counter += 1
            logger.error(f"Error: {e}, waiting 4 seconds and trying again. There was {error_counter} errors so far.")
            time.sleep(4)

if __name__ == "__main__":
    main()

2025-02-15 20:05:35.784 | INFO     | __main__:main:49 - Downloading data...
2025-02-15 20:05:36.337 | INFO     | __main__:main:57 - Got the data
2025-02-15 20:05:40.339 | INFO     | __main__:main:49 - Downloading data...
2025-02-15 20:05:40.821 | INFO     | __main__:main:57 - Got the data
2025-02-15 20:05:44.821 | INFO     | __main__:main:49 - Downloading data...
2025-02-15 20:05:45.326 | INFO     | __main__:main:57 - Got the data
2025-02-15 20:05:49.326 | INFO     | __main__:main:49 - Downloading data...
2025-02-15 20:05:49.819 | INFO     | __main__:main:57 - Got the data
2025-02-15 20:05:53.820 | INFO     | __main__:main:49 - Downloading data...
2025-02-15 20:05:54.349 | INFO     | __main__:main:57 - Got the data
2025-02-15 20:05:58.350 | INFO     | __main__:main:49 - Downloading data...


KeyboardInterrupt: 